In [31]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np


In [32]:
# Masquer les SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [33]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_04.csv")

In [34]:
df_stations_authorized = df[df["is_authorized"] == True]

In [35]:
df_stations_old = df_stations_authorized.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))

412


In [36]:
# filter out station qui n'ont pas de données récentes (2022)
last_year = df["Year"].max()
mask = df_stations_old["Year"]!=last_year
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df_stations_authorized[~df_stations_authorized["NUM_POSTE"].isin(stations_old)]

In [37]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

274


In [38]:
# la précédente exploration nous a permis de conclure que pour prédire à horizon 7 ans
# le modèle ne performait (MAPE moyenne de 13%) qu'à partir de 168 mois (14 ans) de continuité des données
# on filtre donc sur les stations qui ont 12 mois sur chaque année entre 2022 et 2009 inclus au minimum

stations = df_recent["NUM_POSTE"].unique()
to_drop = []

for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(last_year,last_year-14,-1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month = 12 - len(df_station_year["Month"].unique().tolist())
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°1014002 has 12 months in 2022
Station N°1014002 has 12 months in 2021
Station N°1014002 has 12 months in 2020
Station N°1014002 has 12 months in 2019
Station N°1014002 has 12 months in 2018
Station N°1014002 has 12 months in 2017
Station N°1014002 has 12 months in 2016
Station N°1014002 has 12 months in 2015
Station N°1014002 has 12 months in 2014
Station N°1014002 has 12 months in 2013
Station N°1014002 has 12 months in 2012
Station N°1014002 has 12 months in 2011
Station N°1014002 has 12 months in 2010
Station N°1014002 has 12 months in 2009

Station N°1071001 has 12 months in 2022
Station N°1071001 has 12 months in 2021
Station N°1071001 has 12 months in 2020
Station N°1071001 has 12 months in 2019
Station N°1071001 has 12 months in 2018
Station N°1071001 has 12 months in 2017
Station N°1071001 has 12 months in 2016
Station N°1071001 has 12 months in 2015
Station N°1071001 has 12 months in 2014
Station N°1071001 has 12 months in 2013
Station N°1071001 has 12 months in 2012

In [39]:
# gestion des outliers sur la target ?


In [40]:
df_recent = df_recent[~df_recent["NUM_POSTE"].isin(to_drop)]

In [41]:
# 203 stations qui ont au moins 14 années de continuité (12 mois complets) à partir de 2022
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

203


In [42]:
# PREPROCESSING DATE

df_recent["AAAAMM"] = pd.to_datetime(df_recent["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date format AAAAMM et retourne une string format AAAA-MM-DD avec DD = dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_recent["date"] = df_recent["AAAAMM"].apply(last_day_of_month)
df_recent["date"] = pd.to_datetime(df_recent["date"])


In [43]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df if col not in to_keep]

In [44]:
df_recent = df_recent.drop(to_drop,axis=1)

In [45]:
def clean_continuity(dataframe):

    """
    INPUT = un dataframe
    --> parcours la série d'année en année en commençant par la plus récente
    --> si année en cours < 12 mois, drop l'année et toutes les années antérieures
    --> si "trou" entre année en cours N et année suivante, drop N-1 et toutes les années antérieures 
    --> transforme en série temporelle à frequence mensuelle
    OUTPUT = série temporelle clean
    """

    print(f"\nCleaning...")

    #--> check continuité des années : stop_year = N si N < 12 mois, N-1 si "trou" entre N et N-1
    years= df_station["Year"].unique().tolist()
    years.sort(reverse=True)

    month_unique = []
    stop_year = 0

    for index, year in enumerate(years):
        df_station_year = df_station[df_station["Year"] == year]
        month_unique = df_station_year["Month"].unique().tolist()
        if len(month_unique)!=12:
            stop_year = year
            print(f"Année : {year}. Continuité rompue.")
            print(f"Mois: {len(month_unique)} / 12.")
            break
        else:
            try:
                if year - years[index+1] != 1:
                    stop_year = years[index+1]
                    print(f"Année : {years[index+1]}. Continuité rompue.")
                    print(f"Année manquante : {years[index+1]}")
                    break
            except IndexError:
                print("Continuité garantie")
                pass

    #--> drop
    if stop_year > 0:
        dataframe = dataframe[dataframe["Year"] > stop_year]

        print(f"Données antérieures à {stop_year+1} supprimées.")
    else:
        print(f"Aucune rupture de continuité constatée, toutes les données ont été conservées.")
    

    #--> transforme en série temporelle à frequence mensuelle
    dataframe.set_index("date", inplace=True)
    dataframe = dataframe.asfreq("ME")

    #--> check NA
    check = "NOT OK" if dataframe.isna().any().any() else "OK"
    print(f"Check valeurs manquantes : {check}")
    print(f"...Terminé.")

    return dataframe
    

In [46]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df initial en série temporelle + ajout d'une colonne "prédictions"
    """

    print(f"\nForecast à horizon {time} ans...")

    # split variables exogènes / target 
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = data[to_keep]
    y_train = data["vent_speed_inst_moy_mensu"]

    # construction var exog à prédire
    start_pred = var_exog_train.index[-1] + pd.DateOffset(months=1)
    end_pred = start_pred + pd.DateOffset(years=time) - pd.DateOffset(months=1)
    index_pred = pd.date_range(start=start_pred, end=end_pred, freq="ME")

    # les var exog sont constantes, le .max récupère juste la donnée
    var_exog_pred = pd.DataFrame({
    "LON": var_exog_train["LON"].max(),  
    "LAT": var_exog_train["LAT"].max(),   
    "ALTI": var_exog_train["ALTI"].max()
    }, index=index_pred)

    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 2, 12), enforce_stationarity=False)
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time*12, exog = var_exog_pred)
    predicted_values = forecast.predicted_mean

    # rendu
    df_pred = var_exog_pred
    useless_col = ["vent_speed_inst_moy_mensu","Month","Year","LON","LAT","ALTI"] # LON,LAT,ALTI déja dans var_exog_pred, donc dans df_pred à ce stade
    remain_col = [col for col in data.columns if col not in useless_col]
    for col in remain_col:
        df_pred[col] = data[col].max()
    df_pred["pred"] = predicted_values
        
    print("...Terminé.")
    return df_pred


In [47]:
# créé un dictionnaire contenant un dataframe par numéro de station
# clean continuité des dates sur le dataframe + transforme en série temporelle (clean_continuity)
# prédit (get_wind_forecast) et enregistre les prédictions horizon

stations = df_recent["NUM_POSTE"].unique()
horizon = 7

dict_stations = {}

print("##################### START #####################")

for station in stations:
    print(f"\n\n ======> Station : [{station}] <======")
    df_station = df_recent[df_recent["NUM_POSTE"]==station]
    df_station = clean_continuity(df_station)
    dict_stations[station] = get_wind_forecast(horizon,df_station)

print("##################### END #####################")



##################### START #####################


 ======> Station : [1014002] <======

Cleaning...
Année : 2004. Continuité rompue.
Mois: 10 / 12.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [1071001] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [1414001] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 4 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [2037002] <======

Cleaning...
Continuité garantie
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [2173002] <======

Cleaning...


c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...Terminé.


 ======> Station : [15060002] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 7 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [15120005] <======

Cleaning...
Année : 1995. Continuité rompue.
Mois: 7 / 12.
Données antérieures à 1996 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [15122002] <======

Cleaning...
Année : 1992. Continuité rompue.
Mois: 11 / 12.
Données antérieures à 1993 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [16225001] <======

Cleaning...
Année : 1990. Continuité rompue.
Mois: 1 / 12.
Données antérieures à 1991 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7 ans...
...Terminé.


 ======> Station : [17268002] <======

Cleaning...
Année : 1989. Continuité rompue.
Mois: 10 / 12

In [48]:
dict_stations[stations[0]]


,LON,LAT,ALTI,NUM_POSTE,NOM_USUEL,departement_num,departement_name,region,pred
2023-01-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.131917
2023-02-28,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.731668
2023-03-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.977456
2023-04-30,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,7.293580
2023-05-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,7.372916
...,...,...,...,...,...,...,...,...,...
2029-08-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.301794
2029-09-30,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,5.954180
2029-10-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.350162
2029-11-30,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.130554


In [49]:
df_stat = dict_stations[stations[0]].reset_index()

In [50]:
df_pred_total = pd.DataFrame(columns=df_stat.columns)
for key,value in dict_stations.items():
    df_stat = value.reset_index()
    df_pred_total = pd.concat([df_pred_total,df_stat], ignore_index=True)

C:\Users\alexm\AppData\Local\Temp\ipykernel_21436\4082424958.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pred_total = pd.concat([df_pred_total,df_stat], ignore_index=True)


In [51]:
df_pred_total

,index,LON,LAT,ALTI,NUM_POSTE,NOM_USUEL,departement_num,departement_name,region,pred
0,2023-01-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.131917
1,2023-02-28,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.731668
2,2023-03-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,6.977456
3,2023-04-30,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,7.293580
4,2023-05-31,5.669,46.278167,534,1014002,ARBENT,1,Ain,AUVERGNE RHONE ALPES,7.372916
...,...,...,...,...,...,...,...,...,...,...
17047,2029-08-31,1.993,48.526167,116,91200002,DOURDAN,91,Essonne,ILE DE FRANCE,6.451926
17048,2029-09-30,1.993,48.526167,116,91200002,DOURDAN,91,Essonne,ILE DE FRANCE,6.531559
17049,2029-10-31,1.993,48.526167,116,91200002,DOURDAN,91,Essonne,ILE DE FRANCE,6.585855
17050,2029-11-30,1.993,48.526167,116,91200002,DOURDAN,91,Essonne,ILE DE FRANCE,7.408695


In [52]:
# rajouter moyenne force de vent par station

In [53]:
df_pred_total.to_csv(f"pred_horizon_{horizon}_ans.csv", index=False)